# Análisis de Wavelets para Datos MI-EEG

Este notebook implementa análisis avanzado de wavelets para extraer características temporales y espectrales de los datos EEG de imaginación motora.

## Objetivos

1. **Transformada Wavelet Continua (CWT)**: Análisis tiempo-frecuencia usando wavelet de Morlet
2. **Transformada Wavelet Discreta (DWT)**: Descomposición multiresolución usando Daubechies 4
3. **Extracción de características**: Energía por banda, frecuencia dominante, entropía espectral
4. **Preparación para BoF**: Características optimizadas para Bag of Features

## Pipeline

- Carga y procesamiento de datos EEG (independiente del EDA)
- Análisis CWT con escalas logarítmicas
- Análisis DWT con múltiples niveles
- Extracción de características estadísticas
- Visualizaciones tiempo-frecuencia
- Guardado de características para BoF


In [1]:
# Instalación de dependencias (descomentar si es necesario)
# !pip install "numpy>=1.24.0,<2.0.0" mne PyWavelets scikit-learn matplotlib pandas scipy tqdm joblib

# Configuración para ejecución local
import sys
from pathlib import Path

# Directorio raíz del proyecto (local)
ROOT_DIR = Path.cwd()
print(f"Directorio raíz: {ROOT_DIR}")

# Directorios de datos
LEFT_DIR = ROOT_DIR / 'left_imag'
RIGHT_DIR = ROOT_DIR / 'right_imag'

# Directorios de salida (estructura unificada)
RESULTS_DIR = ROOT_DIR / 'results'
DATA_DIR = ROOT_DIR / 'data'

WAVELET_OUTPUT_DIR = RESULTS_DIR / 'wavelets'
BOF_DATA_DIR = DATA_DIR / 'bof_features'
PREPROCESSED_DIR = DATA_DIR / 'preprocessed'

# Crear directorios si no existen
WAVELET_OUTPUT_DIR.mkdir(parents=True, exist_ok=True)
BOF_DATA_DIR.mkdir(parents=True, exist_ok=True)
PREPROCESSED_DIR.mkdir(parents=True, exist_ok=True)

print(f"✅ Directorios configurados:")
print(f"   - Datos Left: {LEFT_DIR}")
print(f"   - Datos Right: {RIGHT_DIR}")
print(f"   - Wavelet Results: {WAVELET_OUTPUT_DIR}")
print(f"   - BoF Features: {BOF_DATA_DIR}")
print(f"   - Preprocessed Data: {PREPROCESSED_DIR}")

Directorio raíz: /Users/manueljurado/Downloads/datos_BCI
✅ Directorios configurados:
   - Datos Left: /Users/manueljurado/Downloads/datos_BCI/left_imag
   - Datos Right: /Users/manueljurado/Downloads/datos_BCI/right_imag
   - Wavelet Results: /Users/manueljurado/Downloads/datos_BCI/results/wavelets
   - BoF Features: /Users/manueljurado/Downloads/datos_BCI/data/bof_features
   - Preprocessed Data: /Users/manueljurado/Downloads/datos_BCI/data/preprocessed


In [2]:
# Celda de instalación de dependencias
# Ejecutar esta celda SOLO si necesitas instalar las librerías en un entorno nuevo

#%pip install mne
#%pip install PyWavelets
#%pip install scikit-learn
#%pip install matplotlib
#%pip install pandas
#%pip install numpy
#%pip install scipy
#%pip install tqdm
#%pip install joblib

print("Instalación de dependencias completada")


Instalación de dependencias completada


In [3]:
# Importar librerías necesarias
import json
import os
import re
from glob import glob
from pathlib import Path
from typing import Dict, List, Tuple, Optional

import matplotlib.pyplot as plt
import mne
import numpy as np
import pandas as pd
import pywt
from mne.io import read_raw_eeglab
from scipy.signal import welch
from sklearn.preprocessing import StandardScaler
from tqdm import tqdm
from joblib import Parallel, delayed
import multiprocessing
import pickle

# Configurar matplotlib
plt.rcParams['figure.figsize'] = (15, 10)
plt.rcParams['font.size'] = 12
plt.style.use('seaborn-v0_8')

# Configuración de paralelización
N_JOBS = max(1, multiprocessing.cpu_count() - 1)  # Usar todos los cores menos uno
print(f"Procesadores disponibles: {multiprocessing.cpu_count()}")
print(f"Cores a utilizar: {N_JOBS}")

print("Librerías importadas correctamente")
print(f"PyWavelets versión: {pywt.__version__}")


Procesadores disponibles: 12
Cores a utilizar: 11
Librerías importadas correctamente
PyWavelets versión: 1.8.0


## Configuración de Parámetros y Carga de Datos

Configuramos los parámetros necesarios y cargamos los datos EEG directamente desde los archivos originales:


## Carga de Datos EEG

Cargamos todos los archivos de datos de imaginación motora (left/right) y los procesamos:


In [4]:
# Parámetros de análisis
LOW_BAND, HIGH_BAND = 8.0, 30.0  # Rango de frecuencias de interés
MU_BAND = (10.0, 12.0)           # Banda mu (ritmo sensoriomotor)
BETA_BAND = (18.0, 26.0)         # Banda beta
EXPECTED_TRIAL_SEC = 9.0         # Duración esperada de cada trial

# Directorios de datos
CANDIDATE_DIRS = {
    "left_imag":  ("left",  "imag"),
    "right_imag": ("right", "imag"),
}

# Regiones cerebrales por prefijos del sistema 10-20
REGION_PREFIXES = {
    "F" : ("Fp", "AF", "F"),    # Frontal
    "FC": ("FC",),              # Frontocentral
    "C" : ("C", "Cz"),          # Central
    "CP": ("CP",),              # Centroparietal
    "P" : ("P",),               # Parietal
    "PO": ("PO",),              # Parietooccipital
    "O" : ("O",),               # Occipital
}

# Funciones auxiliares para carga de datos
def subject_from_fname(fname: str) -> str:
    """Extrae el ID del sujeto del nombre del archivo."""
    m = re.search(r"(S\d{3})", os.path.basename(fname))
    return m.group(1) if m else os.path.basename(fname)

def try_read_epochs(fname: str) -> mne.BaseEpochs:
    """Lee epochs de EEGLAB, creando epochs si es necesario."""
    # 1) Si ya viene epocado
    try:
        ep = mne.read_epochs_eeglab(fname, verbose="ERROR")
        _ = ep.get_data()
        return ep
    except Exception:
        pass

    # 2) Continuo -> ventaneo simple de 9s
    raw = read_raw_eeglab(fname, preload=True, verbose="ERROR")
    try:
        raw.filter(LOW_BAND, HIGH_BAND, verbose="ERROR")
    except Exception:
        pass

    sfreq = float(raw.info["sfreq"])
    n_win = int(np.floor(raw.times[-1] / EXPECTED_TRIAL_SEC))
    if n_win < 1:
        data = np.expand_dims(raw.get_data(), axis=0)
        return mne.EpochsArray(data, raw.info, tmin=0.0, verbose="ERROR")

    picks = mne.pick_types(raw.info, eeg=True, meg=False, stim=False, eog=False)
    samps = int(EXPECTED_TRIAL_SEC * sfreq)
    data_list: List[np.ndarray] = []

    for i in range(n_win):
        s, e = i * samps, (i + 1) * samps
        if e <= raw.n_times:
            data_list.append(np.expand_dims(raw.get_data(picks=picks)[:, s:e], axis=0))

    data = np.concatenate(data_list, axis=0)
    info = mne.create_info([raw.ch_names[p] for p in picks], sfreq, ch_types="eeg")
    return mne.EpochsArray(data, info, tmin=0.0, verbose="ERROR")

print("Parámetros y funciones auxiliares definidas")
print(f"  - Bandas de frecuencia: {LOW_BAND}-{HIGH_BAND} Hz")
print(f"  - Banda μ: {MU_BAND[0]}-{MU_BAND[1]} Hz")
print(f"  - Banda β: {BETA_BAND[0]}-{BETA_BAND[1]} Hz")
print(f"  - Duración por trial: {EXPECTED_TRIAL_SEC} segundos")


Parámetros y funciones auxiliares definidas
  - Bandas de frecuencia: 8.0-30.0 Hz
  - Banda μ: 10.0-12.0 Hz
  - Banda β: 18.0-26.0 Hz
  - Duración por trial: 9.0 segundos


In [5]:
# Cargar epochs de imaginación (left/right) - VERSIÓN LOCAL
epochs_list: List[mne.BaseEpochs] = []
subjects_info = []

# Usar variables definidas en celda 1
print(f"Cargando datos de imaginación motora...")
print(f"  - Left dir: {LEFT_DIR}")
print(f"  - Right dir: {RIGHT_DIR}")

for dirname, task_name in [("left_imag", "left"), ("right_imag", "right")]:
    dpath = Path.cwd() / dirname
    if not dpath.is_dir():
        print(f"⚠️  Advertencia: No existe {dpath}, omitiendo...")
        continue

    print(f"\nProcesando directorio: {dirname}")
    files_processed = 0

    for set_path in sorted(glob(str(dpath / "*.set"))):
        try:
            subject_id = subject_from_fname(set_path)
            ep = try_read_epochs(set_path)
            epochs_list.append(ep)
            subjects_info.append({
                'subject': subject_id,
                'task': task_name,
                'file': str(set_path),
                'n_trials': ep.get_data().shape[0]
            })
            files_processed += 1
            print(f"  {subject_id}: {ep.get_data().shape[0]} trials")
        except Exception as e:
            print(f"  ⚠️  Error en {set_path}: {e}")

print(f"\nResumen de carga:")
print(f"  - Archivos procesados: {len(subjects_info)}")
print(f"  - Total de epochs: {len(epochs_list)}")

if not epochs_list:
    raise ValueError("❌ Error: No se cargaron epochs. Verificar archivos de datos.")
else:
    print("✅ Datos cargados correctamente")


Cargando datos de imaginación motora...
  - Left dir: /Users/manueljurado/Downloads/datos_BCI/left_imag
  - Right dir: /Users/manueljurado/Downloads/datos_BCI/right_imag

Procesando directorio: left_imag
  S001: 22 trials
  S002: 22 trials
  S003: 23 trials
  S004: 23 trials
  S005: 21 trials
  S006: 23 trials
  S007: 22 trials
  S008: 22 trials
  S009: 23 trials
  S010: 23 trials
  S011: 23 trials
  S012: 21 trials
  S013: 22 trials
  S014: 22 trials
  S015: 22 trials
  S016: 21 trials
  S017: 22 trials
  S018: 21 trials
  S019: 22 trials
  S020: 22 trials

Procesando directorio: right_imag
  S001: 22 trials
  S002: 22 trials
  S003: 21 trials
  S004: 21 trials
  S005: 23 trials
  S006: 21 trials
  S007: 22 trials
  S008: 22 trials
  S009: 21 trials
  S010: 21 trials
  S011: 21 trials
  S012: 23 trials
  S013: 22 trials
  S014: 22 trials
  S015: 22 trials
  S016: 23 trials
  S017: 22 trials
  S018: 23 trials
  S019: 22 trials
  S020: 22 trials

Resumen de carga:
  - Archivos procesado

In [6]:
# Configuración de directorios (usar variables ya definidas en celda 1)
data_root = Path(".").resolve()
output_dir = WAVELET_OUTPUT_DIR  # Usar variable global
wavelet_output_dir = WAVELET_OUTPUT_DIR  # Usar variable global

print(f"\nConfiguración de directorios:")
print(f"  - Directorio de datos: {data_root}")
print(f"  - Directorio de salida wavelets: {wavelet_output_dir.resolve()}")



Configuración de directorios:
  - Directorio de datos: /Users/manueljurado/Downloads/datos_BCI
  - Directorio de salida wavelets: /Users/manueljurado/Downloads/datos_BCI/results/wavelets


## Carga de Datos EEG

Cargamos todos los archivos de datos de imaginación motora (left/right) y los procesamos:


## Preparación de Datos

Concatenamos todos los epochs y preparamos los datos para el análisis de wavelets:


In [7]:
# Concatenar epochs y verificar consistencia
base_info = epochs_list[0].info
ch_names = epochs_list[0].ch_names
sfreq = float(epochs_list[0].info["sfreq"])

print(f"\nInformación de los datos:")
print(f"  - Canales: {len(ch_names)}")
print(f"  - Frecuencia de muestreo: {sfreq} Hz")
print(f"  - Primeros 10 canales: {ch_names[:10]}")

# Verificar consistencia entre archivos
for i, ep in enumerate(epochs_list[1:], 1):
    if ep.ch_names != ch_names:
        print(f"Advertencia: Los órdenes de canales difieren en archivo {i}")
    if int(ep.info["sfreq"]) != int(sfreq):
        print(f"Advertencia: sfreq inconsistente en archivo {i}")

# Concatenar todos los datos
X = np.concatenate([ep.get_data() for ep in epochs_list], axis=0)  # (N, ch, T)
N, C, T = X.shape

print(f"\nDatos concatenados:")
print(f"  - Trials totales: {N}")
print(f"  - Canales: {C}")
print(f"  - Muestras por trial: {T}")
print(f"  - Duración por trial: {T/sfreq:.1f} segundos")

# Agrupar canales por regiones cerebrales
ch_upper = [c.upper() for c in ch_names]
region_indices: Dict[str, List[int]] = {}

for reg, prefixes in REGION_PREFIXES.items():
    idxs = []
    for i, c in enumerate(ch_upper):
        if any(c.startswith(p) for p in prefixes):
            idxs.append(i)
    region_indices[reg] = idxs

print(f"\nRegiones identificadas: {list(region_indices.keys())}")
for reg, idxs in region_indices.items():
    if idxs:
        print(f"  - Región {reg}: {len(idxs)} canales")



Información de los datos:
  - Canales: 64
  - Frecuencia de muestreo: 128.0 Hz
  - Primeros 10 canales: ['FC5', 'FC3', 'FC1', 'FCZ', 'FC2', 'FC4', 'FC6', 'C5', 'C3', 'C1']

Datos concatenados:
  - Trials totales: 880
  - Canales: 64
  - Muestras por trial: 1152
  - Duración por trial: 9.0 segundos

Regiones identificadas: ['F', 'FC', 'C', 'CP', 'P', 'PO', 'O']
  - Región F: 26 canales
  - Región FC: 7 canales
  - Región C: 14 canales
  - Región CP: 7 canales
  - Región P: 14 canales
  - Región PO: 5 canales
  - Región O: 3 canales


## Guardado Opcional de Datos Procesados

Opcionalmente, guardamos los datos procesados para uso posterior o compatibilidad con otros notebooks:


In [8]:
# Usar variable global definida en celda 1
shared_data_dir = PREPROCESSED_DIR

# Guardar datos principales
np.save(shared_data_dir / "X_data.npy", X)  # Datos concatenados (trials, channels, time)
np.save(shared_data_dir / "ch_names.npy", ch_names)  # Nombres de canales
np.save(shared_data_dir / "sfreq.npy", np.array([sfreq]))  # Frecuencia de muestreo
np.save(shared_data_dir / "data_dimensions.npy", np.array([N, C, T]))  # Dimensiones

# Guardar información de sujetos
subjects_df = pd.DataFrame(subjects_info)
subjects_df.to_csv(shared_data_dir / "subjects_info.csv", index=False)

# Guardar información de regiones
region_info = {
    'region_indices': region_indices,
    'region_prefixes': REGION_PREFIXES
}
with open(shared_data_dir / "region_info.json", 'w') as f:
    json.dump(region_info, f, indent=2)

# Guardar parámetros de configuración
config_params = {
    'LOW_BAND': LOW_BAND,
    'HIGH_BAND': HIGH_BAND,
    'MU_BAND': list(MU_BAND),
    'BETA_BAND': list(BETA_BAND),
    'EXPECTED_TRIAL_SEC': EXPECTED_TRIAL_SEC,
    'CANDIDATE_DIRS': CANDIDATE_DIRS
}
with open(shared_data_dir / "config_params.json", 'w') as f:
    json.dump(config_params, f, indent=2)

print(f"Datos compartidos guardados en: {shared_data_dir.resolve()}")
print("Archivos generados:")
print(f"  - X_data.npy: Datos concatenados ({X.shape})")
print(f"  - ch_names.npy: Nombres de canales ({len(ch_names)} canales)")
print(f"  - sfreq.npy: Frecuencia de muestreo ({sfreq} Hz)")
print(f"  - data_dimensions.npy: Dimensiones ({N}, {C}, {T})")
print(f"  - subjects_info.csv: Información de sujetos")
print(f"  - region_info.json: Información de regiones")
print(f"  - config_params.json: Parámetros de configuración")

print("\nDatos preparados para análisis de wavelets:")
print(f"  - Datos concatenados: {X.shape} (trials, channels, time)")
print(f"  - Canales: {len(ch_names)}")
print(f"  - Frecuencia de muestreo: {sfreq} Hz")
print(f"  - Dimensiones: {N} trials, {C} canales, {T} muestras")
print(f"  - Duración por trial: {T/sfreq:.1f} segundos")
print(f"  - Sujetos procesados: {len(subjects_info)}")
print(f"  - Regiones identificadas: {list(region_indices.keys())}")


Datos compartidos guardados en: /Users/manueljurado/Downloads/datos_BCI/data/preprocessed
Archivos generados:
  - X_data.npy: Datos concatenados ((880, 64, 1152))
  - ch_names.npy: Nombres de canales (64 canales)
  - sfreq.npy: Frecuencia de muestreo (128.0 Hz)
  - data_dimensions.npy: Dimensiones (880, 64, 1152)
  - subjects_info.csv: Información de sujetos
  - region_info.json: Información de regiones
  - config_params.json: Parámetros de configuración

Datos preparados para análisis de wavelets:
  - Datos concatenados: (880, 64, 1152) (trials, channels, time)
  - Canales: 64
  - Frecuencia de muestreo: 128.0 Hz
  - Dimensiones: 880 trials, 64 canales, 1152 muestras
  - Duración por trial: 9.0 segundos
  - Sujetos procesados: 40
  - Regiones identificadas: ['F', 'FC', 'C', 'CP', 'P', 'PO', 'O']


## Configuración de Wavelets

Definimos los parámetros para el análisis de wavelets:


In [9]:
# Configurar parámetros de wavelets
CWT_SCALES = np.logspace(0.5, 2.5, 50)  # Escalas logarítmicas (1.6-316 Hz aprox)
CWT_WAVELET = 'cmor5.0-1.0'  # Complex Morlet wavelet para análisis tiempo-frecuencia (ancho 5.0, frecuencia central 1.0)
CWT_WIDTH = 5.0  # Ancho de la wavelet de Morlet

# Parámetros DWT
DWT_WAVELET = 'db4'  # Wavelet Daubechies 4
DWT_LEVELS = 6  # Niveles de descomposición

# Bandas de frecuencia de interés para wavelets
FREQ_BANDS = {
    'delta': (1, 4),
    'theta': (4, 8),
    'alpha': (8, 13),
    'beta': (13, 30),
    'gamma': (30, 100)
}

print("Configuración de wavelets:")
print(f"Escalas CWT: {len(CWT_SCALES)} escalas desde {CWT_SCALES[0]:.1f} hasta {CWT_SCALES[-1]:.1f}")
print(f"Wavelet CWT: {CWT_WAVELET} con ancho {CWT_WIDTH}")
print(f"Wavelet DWT: {DWT_WAVELET} con {DWT_LEVELS} niveles")
print(f"Bandas de frecuencia: {list(FREQ_BANDS.keys())}")


Configuración de wavelets:
Escalas CWT: 50 escalas desde 3.2 hasta 316.2
Wavelet CWT: cmor5.0-1.0 con ancho 5.0
Wavelet DWT: db4 con 6 niveles
Bandas de frecuencia: ['delta', 'theta', 'alpha', 'beta', 'gamma']


### Funciones Auxiliares para Wavelets

Definimos las funciones específicas para el análisis de wavelets:


In [10]:
def scales_to_frequencies(scales: np.ndarray, wavelet: str, sampling_rate: float, sort_result: bool = True) -> np.ndarray:
    """Convierte escalas wavelet a frecuencias.

    Si sort_result=True, devuelve frecuencias ordenadas ascendentemente (útil para reporte).
    Si sort_result=False, conserva el orden original de las escalas (útil para enmascarar).
    """
    if wavelet.startswith('cmor'):
        if '-' in wavelet:
            bandwidth = float(wavelet.split('-')[0].replace('cmor', ''))
        else:
            bandwidth = CWT_WIDTH
        frequencies = (bandwidth * sampling_rate) / (2 * np.pi * scales)
    else:
        frequencies = sampling_rate / (2 * scales)
    if sort_result:
        sorted_indices = np.argsort(frequencies)
        return frequencies[sorted_indices]
    return frequencies


def _compute_single_cwt_and_extract_features(args):
    """Calcula CWT y extrae características para una sola señal (usa máscaras precomputadas)."""
    signal, scales, wavelet, sampling_period, original_frequencies, band_masks = args

    # Reducir memoria/tiempo: usar float32 para la señal
    if signal.dtype != np.float32:
        signal = signal.astype(np.float32, copy=False)

    coefficients, _ = pywt.cwt(signal, scales, wavelet, sampling_period=sampling_period)
    power_spectrum = np.abs(coefficients) ** 2  # (n_scales, n_times)

    features = {}

    # 1) Energía por banda
    for band_name, mask in band_masks.items():
        if mask is None or not mask.any():
            features[f'cwt_energy_{band_name}'] = np.nan
            continue
        band_coeffs = coefficients[mask, :]
        features[f'cwt_energy_{band_name}'] = float(np.mean(np.abs(band_coeffs) ** 2))

    # 2) Frecuencia dominante
    power_mean_time = np.mean(power_spectrum, axis=1)
    if power_mean_time.sum() > 1e-10:
        dominant_idx = int(np.argmax(power_mean_time))
        features['cwt_dominant_freq'] = float(original_frequencies[dominant_idx])
    else:
        features['cwt_dominant_freq'] = np.nan

    # 3) Entropía espectral
    power_norm = power_mean_time / (np.sum(power_mean_time) + 1e-10)
    power_norm = power_norm[power_norm > 0]
    spectral_entropy = -np.sum(power_norm * np.log(power_norm + 1e-10))
    features['cwt_spectral_entropy'] = float(spectral_entropy)

    return features


def compute_cwt_features_optimized(data: np.ndarray, sfreq: float, freq_bands: Dict[str, Tuple[float, float]],
                                   n_jobs: int = -1, chunk_size: int = 100,
                                   use_cache: bool = True, cache_dir: Optional[Path] = None) -> Tuple[Dict[str, np.ndarray], np.ndarray]:
    """Calcula coeficientes CWT y extrae características (memoria/CPU optimizado)."""
    n_trials, n_channels, n_times = data.shape

    # Frecuencias de referencia (ordenadas) para reporte
    frequencies_report = scales_to_frequencies(CWT_SCALES, CWT_WAVELET, sfreq, sort_result=True)
    # Frecuencias en el orden original de escalas (para enmascarar y mapeo)
    original_frequencies = scales_to_frequencies(CWT_SCALES, CWT_WAVELET, sfreq, sort_result=False)

    # Cache
    cache_file = None
    if use_cache:
        if cache_dir is None:
            cache_dir = Path("wavelet_reports") / "cwt_features_cache"
        cache_dir.mkdir(parents=True, exist_ok=True)
        import hashlib
        freq_bands_str = json.dumps(freq_bands, sort_keys=True)
        params_hash = hashlib.md5(
            f"{data.shape}_{CWT_SCALES.tobytes()}_{CWT_WAVELET}_{sfreq}_{freq_bands_str}".encode()
        ).hexdigest()[:8]
        cache_file = cache_dir / f"cwt_features_{params_hash}.pkl"
        if cache_file.exists():
            try:
                with open(cache_file, 'rb') as f:
                    cached = pickle.load(f)
                if isinstance(cached, dict) and all(
                    isinstance(v, np.ndarray) and v.shape == (n_trials, n_channels)
                    for v in cached.values()
                ):
                    expected = [f'cwt_energy_{b}' for b in freq_bands.keys()] + ['cwt_dominant_freq', 'cwt_spectral_entropy']
                    if all(k in cached for k in expected):
                        print(f"Cargando características CWT desde cache: {cache_file}")
                        return cached, frequencies_report
            except Exception as e:
                print(f"Error al cargar cache: {e}. Recalculando...")

    # n_jobs
    if n_jobs == -1:
        try:
            n_jobs = N_JOBS
        except NameError:
            n_jobs = max(1, multiprocessing.cpu_count() - 1)

    print("Calculando coeficientes CWT y extrayendo características (optimizado)...")
    print(f"  - Total de señales a procesar: {n_trials * n_channels}")
    print(f"  - Procesadores a utilizar: {n_jobs if n_jobs > 0 else 'todos'}")
    print(f"  - Chunk size: {chunk_size}")

    # Precomputar máscaras de bandas sobre frecuencias en orden original
    band_masks: Dict[str, Optional[np.ndarray]] = {}
    for band_name, (fmin, fmax) in freq_bands.items():
        mask = (original_frequencies >= fmin) & (original_frequencies <= fmax)
        band_masks[band_name] = mask if mask.any() else None

    # Preparar argumentos
    data_flat = data.reshape(n_trials * n_channels, n_times)
    args_list = [
        (data_flat[i, :], CWT_SCALES, CWT_WAVELET, 1.0 / sfreq, original_frequencies, band_masks)
        for i in range(n_trials * n_channels)
    ]

    results = Parallel(n_jobs=n_jobs, backend='threading', batch_size=chunk_size)(
        delayed(_compute_single_cwt_and_extract_features)(args)
        for args in tqdm(args_list, desc="Procesando CWT y extrayendo características")
    )

    # Ensamblar resultados sin DataFrame
    feature_lists: Dict[str, List[float]] = {}
    for res in results:
        for k, v in res.items():
            feature_lists.setdefault(k, []).append(v)

    extracted_features: Dict[str, np.ndarray] = {}
    for k, vals in feature_lists.items():
        extracted_features[k] = np.asarray(vals, dtype=np.float32).reshape(n_trials, n_channels)

    if use_cache and cache_file is not None:
        try:
            with open(cache_file, 'wb') as f:
                pickle.dump(extracted_features, f)
            print(f"Características CWT guardadas en cache: {cache_file}")
        except Exception as e:
            print(f"Error al guardar cache: {e}")

    return extracted_features, frequencies_report


print("Funciones auxiliares para wavelets definidas (versión optimizada)")
print(f"PyWavelets versión: {pywt.__version__}")


Funciones auxiliares para wavelets definidas (versión optimizada)
PyWavelets versión: 1.8.0


## Análisis CWT (Transformada Wavelet Continua)

Calculamos los coeficientes CWT para análisis tiempo-frecuencia:


In [11]:
# Calcular coeficientes CWT y extraer características usando la función optimizada
print("Iniciando análisis CWT y extracción de características optimizada...")
print(f"Datos de entrada: {X.shape} (trials, channels, time)")
print(f"Frecuencia de muestreo: {sfreq} Hz")
print(f"Bandas de frecuencia para extracción: {list(FREQ_BANDS.keys())}")


# Las características CWT más importantes para BoF se extraen directamente aquí
# Definimos las características clave que queremos obtener de CWT para el paso de combinación
cwt_key_features_to_extract = {
    'cwt_energy_delta': FREQ_BANDS['delta'],
    'cwt_energy_theta': FREQ_BANDS['theta'],
    'cwt_energy_alpha': FREQ_BANDS['alpha'],
    'cwt_energy_beta': FREQ_BANDS['beta'],
    'cwt_energy_gamma': FREQ_BANDS['gamma'],
    # Frecuencia dominante y entropía espectral se extraen si es posible
    # No necesitan una banda de frecuencia específica, se calculan sobre todas las escalas
}


cwt_features, cwt_frequencies = compute_cwt_features_optimized(
    X, sfreq,
    freq_bands=FREQ_BANDS, # Pasar las bandas de frecuencia para extracción
    n_jobs=-1,           # Usar procesamiento paralelo (-1 = N_JOBS global)
    chunk_size=50,       # Procesar 50 señales por lote
    use_cache=True       # Habilitar cache
)

print(f"\nAnálisis CWT y extracción de características completados:")
print(f"Características CWT extraídas:")
for feature_name, feature_array in cwt_features.items():
    print(f"  - {feature_name}: {feature_array.shape}")

print(f"Frecuencias CWT: {len(cwt_frequencies)} puntos")
print(f"Rango de frecuencias: {cwt_frequencies[0]:.1f} - {cwt_frequencies[-1]:.1f} Hz")


# Mostrar información sobre las bandas de frecuencia (basado en las frecuencias calculadas)
print(f"\nBandas de frecuencia CWT (basado en escalas y sfreq):")
for band_name, (fmin, fmax) in FREQ_BANDS.items():
    # Usar cwt_frequencies que ya está ordenado
    mask = (cwt_frequencies >= fmin) & (cwt_frequencies <= fmax)
    n_scales = mask.sum()
    if n_scales > 0:
        print(f"  - {band_name} ({fmin}-{fmax} Hz): {n_scales} escalas")
    else:
        print(f"  - {band_name} ({fmin}-{fmax} Hz): fuera del rango")


Iniciando análisis CWT y extracción de características optimizada...
Datos de entrada: (880, 64, 1152) (trials, channels, time)
Frecuencia de muestreo: 128.0 Hz
Bandas de frecuencia para extracción: ['delta', 'theta', 'alpha', 'beta', 'gamma']
Calculando coeficientes CWT y extrayendo características (optimizado)...
  - Total de señales a procesar: 56320
  - Procesadores a utilizar: 11
  - Chunk size: 50


Procesando CWT y extrayendo características: 100%|██████████| 56320/56320 [06:45<00:00, 138.78it/s]


Características CWT guardadas en cache: wavelet_reports/cwt_features_cache/cwt_features_d19cc8dc.pkl

Análisis CWT y extracción de características completados:
Características CWT extraídas:
  - cwt_energy_delta: (880, 64)
  - cwt_energy_theta: (880, 64)
  - cwt_energy_alpha: (880, 64)
  - cwt_energy_beta: (880, 64)
  - cwt_energy_gamma: (880, 64)
  - cwt_dominant_freq: (880, 64)
  - cwt_spectral_entropy: (880, 64)
Frecuencias CWT: 50 puntos
Rango de frecuencias: 0.3 - 32.2 Hz

Bandas de frecuencia CWT (basado en escalas y sfreq):
  - delta (1-4 Hz): 14 escalas
  - theta (4-8 Hz): 8 escalas
  - alpha (8-13 Hz): 5 escalas
  - beta (13-30 Hz): 9 escalas
  - gamma (30-100 Hz): 1 escalas


## Análisis DWT (Transformada Wavelet Discreta)

Implementamos análisis DWT para descomposición multiresolución:


In [12]:
def compute_dwt_coefficients(data: np.ndarray, wavelet: str, levels: int) -> Dict[str, np.ndarray]:
    """
    Calcula coeficientes DWT para todos los canales y trials.

    Args:
        data: Array de forma (trials, channels, time)
        wavelet: Tipo de wavelet (ej. 'db4')
        levels: Número de niveles de descomposición

    Returns:
        Diccionario con coeficientes DWT por nivel
    """
    n_trials, n_channels, n_times = data.shape

    # Inicializar diccionario para coeficientes
    dwt_coeffs = {}

    print("Calculando coeficientes DWT...")
    for trial_idx in tqdm(range(n_trials), desc="Trials"):
        for ch_idx in range(n_channels):
            signal = data[trial_idx, ch_idx, :]

            # Calcular DWT
            coeffs = pywt.wavedec(signal, wavelet, level=levels)

            # Guardar coeficientes por nivel
            for level, coeff in enumerate(coeffs):
                key = f'level_{level}'
                if key not in dwt_coeffs:
                    dwt_coeffs[key] = np.zeros((n_trials, n_channels, len(coeff)))
                dwt_coeffs[key][trial_idx, ch_idx, :] = coeff

    return dwt_coeffs

# Calcular coeficientes DWT
print("Iniciando análisis DWT...")
print(f"Datos de entrada: {X.shape} (trials, channels, time)")
print(f"Wavelet: {DWT_WAVELET}, Niveles: {DWT_LEVELS}")

dwt_coeffs = compute_dwt_coefficients(X, DWT_WAVELET, DWT_LEVELS)

print(f"\nDWT completado:")
print(f"Coeficientes DWT por nivel:")
for level_key, coeff_array in dwt_coeffs.items():
    print(f"  - {level_key}: {coeff_array.shape}")


Iniciando análisis DWT...
Datos de entrada: (880, 64, 1152) (trials, channels, time)
Wavelet: db4, Niveles: 6
Calculando coeficientes DWT...


Trials: 100%|██████████| 880/880 [00:06<00:00, 145.14it/s]


DWT completado:
Coeficientes DWT por nivel:
  - level_0: (880, 64, 24)
  - level_1: (880, 64, 24)
  - level_2: (880, 64, 42)
  - level_3: (880, 64, 78)
  - level_4: (880, 64, 150)
  - level_5: (880, 64, 293)
  - level_6: (880, 64, 579)


### Extracción de Características DWT

Extraemos características estadísticas de los coeficientes DWT:


In [13]:
def extract_dwt_features(dwt_coeffs: Dict[str, np.ndarray]) -> Dict[str, np.ndarray]:
    """
    Extrae características de los coeficientes DWT.

    Args:
        dwt_coeffs: Diccionario con coeficientes DWT por nivel

    Returns:
        Diccionario con características extraídas
    """
    features = {}

    print("Extrayendo características DWT...")

    # 1. Energía por nivel de descomposición
    for level_key, coeff_array in dwt_coeffs.items():
        # Energía promedio por trial y canal
        energy = np.mean(coeff_array**2, axis=2)  # (trials, channels)
        features[f'dwt_energy_{level_key}'] = energy
        print(f"  - Energía {level_key}: {energy.shape}")

    # 2. Estadísticas por nivel
    for level_key, coeff_array in dwt_coeffs.items():
        # Media por trial y canal
        mean_coeffs = np.mean(coeff_array, axis=2)  # (trials, channels)
        features[f'dwt_mean_{level_key}'] = mean_coeffs

        # Desviación estándar por trial y canal
        std_coeffs = np.std(coeff_array, axis=2)  # (trials, channels)
        features[f'dwt_std_{level_key}'] = std_coeffs

        # Máximo absoluto por trial y canal
        max_coeffs = np.max(np.abs(coeff_array), axis=2)  # (trials, channels)
        features[f'dwt_max_{level_key}'] = max_coeffs

        print(f"  - Estadísticas {level_key}: media, std, max")

    # 3. Relación de energía entre niveles (aproximación de bandas de frecuencia)
    if 'level_0' in dwt_coeffs and 'level_1' in dwt_coeffs:
        # Relación entre aproximación y detalle
        energy_level_0 = np.mean(dwt_coeffs['level_0']**2, axis=2)
        energy_level_1 = np.mean(dwt_coeffs['level_1']**2, axis=2)
        energy_ratio = energy_level_0 / (energy_level_1 + 1e-10)
        features['dwt_energy_ratio_level0_level1'] = energy_ratio
        print(f"  - Relación de energía level0/level1: {energy_ratio.shape}")

    return features

# Extraer características DWT
dwt_features = extract_dwt_features(dwt_coeffs)

print(f"\nCaracterísticas DWT extraídas:")
for feature_name, feature_array in dwt_features.items():
    print(f"  - {feature_name}: {feature_array.shape}")


Extrayendo características DWT...
  - Energía level_0: (880, 64)
  - Energía level_1: (880, 64)
  - Energía level_2: (880, 64)
  - Energía level_3: (880, 64)
  - Energía level_4: (880, 64)
  - Energía level_5: (880, 64)
  - Energía level_6: (880, 64)
  - Estadísticas level_0: media, std, max
  - Estadísticas level_1: media, std, max
  - Estadísticas level_2: media, std, max
  - Estadísticas level_3: media, std, max
  - Estadísticas level_4: media, std, max
  - Estadísticas level_5: media, std, max
  - Estadísticas level_6: media, std, max
  - Relación de energía level0/level1: (880, 64)

Características DWT extraídas:
  - dwt_energy_level_0: (880, 64)
  - dwt_energy_level_1: (880, 64)
  - dwt_energy_level_2: (880, 64)
  - dwt_energy_level_3: (880, 64)
  - dwt_energy_level_4: (880, 64)
  - dwt_energy_level_5: (880, 64)
  - dwt_energy_level_6: (880, 64)
  - dwt_mean_level_0: (880, 64)
  - dwt_std_level_0: (880, 64)
  - dwt_max_level_0: (880, 64)
  - dwt_mean_level_1: (880, 64)
  - dwt_st

## Preparación de Características para BoF

Combinamos todas las características extraídas y las preparamos para Bag of Features:


In [14]:
# Combinar todas las características
print("Combinando características para BoF...")

# Combinar características CWT y DWT
all_features = {}
all_features.update(cwt_features)
all_features.update(dwt_features)

print(f"Total de características extraídas: {len(all_features)}")

# Crear matriz de características para BoF
feature_names = []
feature_matrices = []

for feature_name, feature_array in all_features.items():
    # Reshape para tener una matriz 2D (samples, features)
    if len(feature_array.shape) == 2:  # (trials, channels)
        # Cada canal es una característica
        for ch_idx, ch_name in enumerate(ch_names):
            feature_names.append(f"{feature_name}_{ch_name}")
            feature_matrices.append(feature_array[:, ch_idx])
    elif len(feature_array.shape) == 3:  # (trials, channels, scales/levels)
        # Cada combinación canal-escala es una característica
        for ch_idx, ch_name in enumerate(ch_names):
            for scale_idx in range(feature_array.shape[2]):
                feature_names.append(f"{feature_name}_{ch_name}_scale{scale_idx}")
                feature_matrices.append(feature_array[:, ch_idx, scale_idx])

# Crear matriz final de características
X_features = np.column_stack(feature_matrices)  # (trials, total_features)

print(f"Matriz de características final:")
print(f"  - Trials: {X_features.shape[0]}")
print(f"  - Características totales: {X_features.shape[1]}")
print(f"  - Primeras 10 características: {feature_names[:10]}")

# Imputar NaNs por la media de cada columna antes de normalizar
if np.isnan(X_features).any():
    col_means = np.nanmean(X_features, axis=0)
    inds = np.where(np.isnan(X_features))
    X_features[inds] = np.take(col_means, inds[1])

# Normalizar características
scaler = StandardScaler()
X_features_normalized = scaler.fit_transform(X_features)

print(f"Características normalizadas: {X_features_normalized.shape}")
print(f"Media después de normalización: {np.mean(X_features_normalized):.6f}")
print(f"Desviación estándar después de normalización: {np.std(X_features_normalized):.6f}")


Combinando características para BoF...
Total de características extraídas: 36
Matriz de características final:
  - Trials: 880
  - Características totales: 2304
  - Primeras 10 características: ['cwt_energy_delta_FC5', 'cwt_energy_delta_FC3', 'cwt_energy_delta_FC1', 'cwt_energy_delta_FCZ', 'cwt_energy_delta_FC2', 'cwt_energy_delta_FC4', 'cwt_energy_delta_FC6', 'cwt_energy_delta_C5', 'cwt_energy_delta_C3', 'cwt_energy_delta_C1']
Características normalizadas: (880, 2304)
Media después de normalización: 0.000000
Desviación estándar después de normalización: 1.000000


## Guardado de Archivos para BoF

Guardamos las características y metadatos necesarios para implementar Bag of Features:


In [16]:
# Guardar características normalizadas para BoF
features_file = wavelet_output_dir / "wavelet_features.npy"
np.save(features_file, X_features_normalized)
print(f"Características guardadas: {features_file.resolve()}")

# Guardar nombres de características
feature_names_file = wavelet_output_dir / "feature_names.txt"
with open(feature_names_file, 'w') as f:
    for name in feature_names:
        f.write(f"{name}\n")
print(f"Nombres de características guardados: {feature_names_file.resolve()}")

# Guardar información de canales
channel_info = pd.DataFrame({
    'channel_index': range(len(ch_names)),
    'channel_name': ch_names,
    'region': ['unknown'] * len(ch_names)  # Se puede mejorar con mapeo de regiones
})

# Mapear regiones basado en prefijos
for idx, ch_name in enumerate(ch_names):
    ch_upper = ch_name.upper()
    for region, prefixes in REGION_PREFIXES.items():
        if any(ch_upper.startswith(p) for p in prefixes):
            channel_info.loc[idx, 'region'] = region
            break

channel_info_file = wavelet_output_dir / "channel_info.csv"
channel_info.to_csv(channel_info_file, index=False)
print(f"Información de canales guardada: {channel_info_file.resolve()}")

# Guardar información de sujetos y tareas
subjects_df = pd.DataFrame(subjects_info)
subjects_file = wavelet_output_dir / "subjects_info.csv"
subjects_df.to_csv(subjects_file, index=False)
print(f"Información de sujetos guardada: {subjects_file.resolve()}")

# Guardar parámetros de configuración
config_info = {
    'cwt_scales': CWT_SCALES.tolist(),
    'cwt_wavelet': CWT_WAVELET,
    'cwt_width': CWT_WIDTH,
    'dwt_wavelet': DWT_WAVELET,
    'dwt_levels': DWT_LEVELS,
    'freq_bands': FREQ_BANDS,
    'sampling_rate': sfreq,
    'n_trials': N,
    'n_channels': C,
    'n_timepoints': T,
    'feature_dimensions': X_features_normalized.shape[1]
}

config_file = wavelet_output_dir / "wavelet_config.json"
import json
with open(config_file, 'w') as f:
    json.dump(config_info, f, indent=2)
print(f"Configuración guardada: {config_file.resolve()}")

print(f"\nResumen de archivos generados:")
print(f"  - {features_file.name}: Características normalizadas ({X_features_normalized.shape})")
print(f"  - {feature_names_file.name}: Nombres de características ({len(feature_names)} características)")
print(f"  - {channel_info_file.name}: Información de canales ({len(ch_names)} canales)")
print(f"  - {subjects_file.name}: Información de sujetos ({len(subjects_info)} archivos)")
print(f"  - {config_file.name}: Parámetros de configuración")


Características guardadas: /Users/manueljurado/Downloads/datos_BCI/results/wavelets/wavelet_features.npy
Nombres de características guardados: /Users/manueljurado/Downloads/datos_BCI/results/wavelets/feature_names.txt
Información de canales guardada: /Users/manueljurado/Downloads/datos_BCI/results/wavelets/channel_info.csv
Información de sujetos guardada: /Users/manueljurado/Downloads/datos_BCI/results/wavelets/subjects_info.csv
Configuración guardada: /Users/manueljurado/Downloads/datos_BCI/results/wavelets/wavelet_config.json

Resumen de archivos generados:
  - wavelet_features.npy: Características normalizadas ((880, 2304))
  - feature_names.txt: Nombres de características (2304 características)
  - channel_info.csv: Información de canales (64 canales)
  - subjects_info.csv: Información de sujetos (40 archivos)
  - wavelet_config.json: Parámetros de configuración


## Preparación de Datos para Bag of Features (BoF)

Esta sección prepara específicamente los datos que necesita el algoritmo Bag of Features para clasificación:


In [17]:
# Usar variable global definida en celda 1 (usado por 03_BoF_Clasificacion.ipynb)
bof_data_dir = BOF_DATA_DIR

print("Preparando datos específicos para Bag of Features...")
print(f"Directorio BoF: {bof_data_dir.resolve()}")

# 1. Crear etiquetas de clase basadas en la tarea (left/right)
print("\n1. Creando etiquetas de clase...")

# Crear array de etiquetas basado en subjects_info
y_labels = []
trial_to_subject = []  # Mapeo de trial a sujeto
trial_to_task = []     # Mapeo de trial a tarea

trial_idx = 0
for subject_info in subjects_info:
    n_trials = subject_info['n_trials']
    task = subject_info['task']
    subject = subject_info['subject']

    # Etiquetas: 0 = left, 1 = right
    label = 0 if task == 'left' else 1

    for _ in range(n_trials):
        y_labels.append(label)
        trial_to_subject.append(subject)
        trial_to_task.append(task)
        trial_idx += 1

y_labels = np.array(y_labels)
print(f"  - Etiquetas creadas: {y_labels.shape}")
print(f"  - Clase 0 (left): {np.sum(y_labels == 0)} trials")
print(f"  - Clase 1 (right): {np.sum(y_labels == 1)} trials")

# Guardar etiquetas
np.save(bof_data_dir / "y_labels.npy", y_labels)
np.save(bof_data_dir / "trial_to_subject.npy", np.array(trial_to_subject))
np.save(bof_data_dir / "trial_to_task.npy", np.array(trial_to_task))

print(f"  - Etiquetas guardadas en: {bof_data_dir / 'y_labels.npy'}")


Preparando datos específicos para Bag of Features...
Directorio BoF: /Users/manueljurado/Downloads/datos_BCI/data/bof_features

1. Creando etiquetas de clase...
  - Etiquetas creadas: (880,)
  - Clase 0 (left): 442 trials
  - Clase 1 (right): 438 trials
  - Etiquetas guardadas en: /Users/manueljurado/Downloads/datos_BCI/data/bof_features/y_labels.npy


In [18]:
# 2. Preparar características específicas para BoF
print("\n2. Preparando características para BoF...")

# Seleccionar características más relevantes para BoF
selected_features = {}

# Características CWT más importantes
cwt_key_features = [
    'cwt_energy_alpha',    # Energía en banda alpha (8-13 Hz)
    'cwt_energy_beta',    # Energía en banda beta (13-30 Hz)
    'cwt_dominant_freq',  # Frecuencia dominante
    'cwt_spectral_entropy' # Entropía espectral
]

for feature_name in cwt_key_features:
    if feature_name in cwt_features:
        selected_features[feature_name] = cwt_features[feature_name]
        print(f"  - {feature_name}: {cwt_features[feature_name].shape}")

# Características DWT más importantes (primeros 3 niveles)
dwt_key_features = [
    'dwt_energy_level_0',  # Aproximación (baja frecuencia)
    'dwt_energy_level_1',  # Primer detalle
    'dwt_energy_level_2',  # Segundo detalle
    'dwt_mean_level_0',    # Media de aproximación
    'dwt_std_level_0'      # Desviación estándar de aproximación
]

for feature_name in dwt_key_features:
    if feature_name in dwt_features:
        selected_features[feature_name] = dwt_features[feature_name]
        print(f"  - {feature_name}: {dwt_features[feature_name].shape}")

print(f"\nTotal de características seleccionadas: {len(selected_features)}")

# Crear matriz de características seleccionadas
bof_feature_names = []
bof_feature_matrices = []

for feature_name, feature_array in selected_features.items():
    if len(feature_array.shape) == 2:  # (trials, channels)
        for ch_idx, ch_name in enumerate(ch_names):
            bof_feature_names.append(f"{feature_name}_{ch_name}")
            bof_feature_matrices.append(feature_array[:, ch_idx])
    elif len(feature_array.shape) == 3:  # (trials, channels, scales)
        for ch_idx, ch_name in enumerate(ch_names):
            for scale_idx in range(feature_array.shape[2]):
                bof_feature_names.append(f"{feature_name}_{ch_name}_scale{scale_idx}")
                bof_feature_matrices.append(feature_array[:, ch_idx, scale_idx])

# Crear matriz final de características BoF
X_bof = np.column_stack(bof_feature_matrices)

print(f"Matriz de características BoF:")
print(f"  - Trials: {X_bof.shape[0]}")
print(f"  - Características: {X_bof.shape[1]}")
print(f"  - Primeras 10 características: {bof_feature_names[:10]}")

# Imputar NaNs por la media de cada columna antes de normalizar
if np.isnan(X_bof).any():
    col_means_bof = np.nanmean(X_bof, axis=0)
    inds_bof = np.where(np.isnan(X_bof))
    X_bof[inds_bof] = np.take(col_means_bof, inds_bof[1])

# Normalizar características BoF
scaler_bof = StandardScaler()
X_bof_normalized = scaler_bof.fit_transform(X_bof)

print(f"Características BoF normalizadas: {X_bof_normalized.shape}")
print(f"Media después de normalización: {np.mean(X_bof_normalized):.6f}")
print(f"Desviación estándar: {np.std(X_bof_normalized):.6f}")



2. Preparando características para BoF...
  - cwt_energy_alpha: (880, 64)
  - cwt_energy_beta: (880, 64)
  - cwt_dominant_freq: (880, 64)
  - cwt_spectral_entropy: (880, 64)
  - dwt_energy_level_0: (880, 64)
  - dwt_energy_level_1: (880, 64)
  - dwt_energy_level_2: (880, 64)
  - dwt_mean_level_0: (880, 64)
  - dwt_std_level_0: (880, 64)

Total de características seleccionadas: 9
Matriz de características BoF:
  - Trials: 880
  - Características: 576
  - Primeras 10 características: ['cwt_energy_alpha_FC5', 'cwt_energy_alpha_FC3', 'cwt_energy_alpha_FC1', 'cwt_energy_alpha_FCZ', 'cwt_energy_alpha_FC2', 'cwt_energy_alpha_FC4', 'cwt_energy_alpha_FC6', 'cwt_energy_alpha_C5', 'cwt_energy_alpha_C3', 'cwt_energy_alpha_C1']
Características BoF normalizadas: (880, 576)
Media después de normalización: 0.000000
Desviación estándar: 1.000000


In [19]:
# 3. Guardar datos específicos para BoF
print("\n3. Guardando datos específicos para BoF...")

# Guardar características BoF
np.save(bof_data_dir / "X_bof_features.npy", X_bof_normalized)
print(f"  - Características BoF: {bof_data_dir / 'X_bof_features.npy'}")

# Guardar nombres de características BoF
with open(bof_data_dir / "bof_feature_names.txt", 'w') as f:
    for name in bof_feature_names:
        f.write(f"{name}\n")
print(f"  - Nombres de características: {bof_data_dir / 'bof_feature_names.txt'}")

# Guardar normalizador BoF
import pickle
with open(bof_data_dir / "scaler_bof.pkl", 'wb') as f:
    pickle.dump(scaler_bof, f)
print(f"  - Normalizador BoF: {bof_data_dir / 'scaler_bof.pkl'}")

# Crear información de metadatos para BoF
bof_metadata = {
    'n_trials': X_bof_normalized.shape[0],
    'n_features': X_bof_normalized.shape[1],
    'n_channels': len(ch_names),
    'n_subjects': len(subjects_info),
    'class_distribution': {
        'left_trials': int(np.sum(y_labels == 0)),
        'right_trials': int(np.sum(y_labels == 1))
    },
    'feature_types': {
        'cwt_features': len([f for f in bof_feature_names if f.startswith('cwt_')]),
        'dwt_features': len([f for f in bof_feature_names if f.startswith('dwt_')])
    },
    'sampling_rate': sfreq,
    'trial_duration_sec': T / sfreq,
    'wavelet_config': {
        'cwt_scales': len(CWT_SCALES),
        'cwt_wavelet': CWT_WAVELET,
        'dwt_levels': DWT_LEVELS,
        'dwt_wavelet': DWT_WAVELET
    }
}

with open(bof_data_dir / "bof_metadata.json", 'w') as f:
    json.dump(bof_metadata, f, indent=2)
print(f"  - Metadatos BoF: {bof_data_dir / 'bof_metadata.json'}")

# Crear archivo de configuración para BoF
bof_config = {
    'data_files': {
        'features': 'X_bof_features.npy',
        'labels': 'y_labels.npy',
        'feature_names': 'bof_feature_names.txt',
        'scaler': 'scaler_bof.pkl',
        'metadata': 'bof_metadata.json'
    },
    'recommended_params': {
        'n_clusters': [50, 100, 200],  # Número de clusters para BoF
        'random_state': 42,
        'test_size': 0.2,
        'cv_folds': 5
    },
    'feature_info': {
        'total_features': len(bof_feature_names),
        'cwt_features': len([f for f in bof_feature_names if f.startswith('cwt_')]),
        'dwt_features': len([f for f in bof_feature_names if f.startswith('dwt_')]),
        'normalized': True,
        'scaler_type': 'StandardScaler'
    }
}

with open(bof_data_dir / "bof_config.json", 'w') as f:
    json.dump(bof_config, f, indent=2)
print(f"  - Configuración BoF: {bof_data_dir / 'bof_config.json'}")

print(f"\nDatos BoF guardados exitosamente en: {bof_data_dir.resolve()}")



3. Guardando datos específicos para BoF...
  - Características BoF: /Users/manueljurado/Downloads/datos_BCI/data/bof_features/X_bof_features.npy
  - Nombres de características: /Users/manueljurado/Downloads/datos_BCI/data/bof_features/bof_feature_names.txt
  - Normalizador BoF: /Users/manueljurado/Downloads/datos_BCI/data/bof_features/scaler_bof.pkl
  - Metadatos BoF: /Users/manueljurado/Downloads/datos_BCI/data/bof_features/bof_metadata.json
  - Configuración BoF: /Users/manueljurado/Downloads/datos_BCI/data/bof_features/bof_config.json

Datos BoF guardados exitosamente en: /Users/manueljurado/Downloads/datos_BCI/data/bof_features


## Resumen del Análisis de Wavelets y Preparación BoF

### Análisis Completados

1. **Transformada Wavelet Continua (CWT)**
   - Wavelet de Morlet con 50 escalas logarítmicas
   - Análisis tiempo-frecuencia completo
   - Extracción de energía por banda, frecuencia dominante y entropía espectral

2. **Transformada Wavelet Discreta (DWT)**
   - Wavelet Daubechies 4 con 6 niveles de descomposición
   - Análisis multiresolución
   - Extracción de estadísticas por nivel (energía, media, desviación estándar, máximo)

3. **Preparación Específica para BoF**
   - Selección de características más relevantes
   - Normalización específica para BoF
   - Etiquetas de clase (left/right)
   - Metadatos completos y configuración

### Archivos Generados para BoF

Todos los archivos específicos para BoF se guardaron en el directorio `bof_data/`:

- **`X_bof_features.npy`**: Características normalizadas específicas para BoF
- **`y_labels.npy`**: Etiquetas de clase (0=left, 1=right)
- **`bof_feature_names.txt`**: Nombres de características seleccionadas
- **`scaler_bof.pkl`**: Normalizador entrenado para BoF
- **`bof_metadata.json`**: Metadatos completos del dataset
- **`bof_config.json`**: Configuración y parámetros recomendados
- **`trial_to_subject.npy`**: Mapeo de trials a sujetos
- **`trial_to_task.npy`**: Mapeo de trials a tareas

### Características Seleccionadas para BoF

**CWT Features:**
- Energía en banda alpha (8-13 Hz)
- Energía en banda beta (13-30 Hz)
- Frecuencia dominante por canal
- Entropía espectral por canal

**DWT Features:**
- Energía de aproximación (nivel 0)
- Energía de detalles (niveles 1-2)
- Media y desviación estándar de aproximación

### Próximos Pasos para BoF

Los datos están completamente preparados para implementar Bag of Features:

1. **Cargar datos**: Usar archivos en `bof_data/`
2. **Clustering**: Aplicar K-means con parámetros recomendados
3. **Codificación**: Crear histogramas de características por trial
4. **Clasificación**: Entrenar clasificadores SVM/Random Forest
5. **Evaluación**: Validación cruzada y métricas de rendimiento

### Variables Disponibles para BoF

- `X_bof_normalized`: Características normalizadas para BoF
- `y_labels`: Etiquetas de clase
- `bof_feature_names`: Nombres de características
- `scaler_bof`: Normalizador entrenado
- `bof_metadata`: Metadatos del dataset
- `bof_config`: Configuración recomendada
